In [1]:
import numpy as np
import pandas as pd

import json
import keras
import tensorflow
import tensorflow_addons as tfa
from keras import layers, Model
from keras.models import Sequential
from keras.applications import DenseNet201
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from tqdm import tqdm

Using TensorFlow backend.


In [2]:
csv_path = './UrbanSound8K/metadata/UrbanSound8K.csv'
spec_path = "./numpySpectrograms/"
model_save_path = "ovr_basemodel"
test_size = 0.2
val_size = 0.2
batch_size = 16
num_classes = 5

In [3]:
class SpecLoader(keras.utils.Sequence):
  def __init__(self, x_set, y_set, batch_size, spec_dir):
    self.x, self.y = x_set, y_set
    self.batch_size = batch_size
    self.spec_dir = spec_dir

  def __len__(self):
    return int(np.ceil(len(self.x) / self.batch_size))

  def __getitem__(self, idx):
    batch_x = self.x[idx * self.batch_size:(idx + 1) *
    self.batch_size]
    batch_y = self.y[idx * self.batch_size:(idx + 1) *
        self.batch_size]

    batchSpecs = []
    for fileName in batch_x:
        spec = np.load(self.spec_dir + fileName + ".npy")
        batchSpecs.append(spec.transpose())
    return np.array(batchSpecs), np.array(batch_y)

In [4]:
data_df = pd.read_csv(csv_path)
data_df_known = data_df.loc[data_df["classID"] < num_classes]
data_df_unknown = data_df.loc[data_df["classID"] >= num_classes]

X_trainval, X_test, y_trainval, y_test = train_test_split(data_df_known['slice_file_name'].tolist(), data_df_known['classID'].tolist(), test_size=test_size, random_state = 42)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=val_size, random_state = 42)
X_trash, X_unknown, y_trash, y_unknown = train_test_split(data_df_unknown['slice_file_name'].tolist(), data_df_unknown['classID'].tolist(), test_size=test_size, random_state = 42)
y_unknown = [num_classes] * len(y_unknown)

X_test = X_test + X_unknown
y_test = y_test + y_unknown
test_loader = SpecLoader(X_test, y_test, batch_size, spec_path)

In [5]:
models = []
for i in range(num_classes):
    models.append(keras.models.load_model(model_save_path+str(i)))

In [6]:
def evaluate(predicted, expected):
    acc = np.mean(np.array(predicted) == np.array(expected))
    print("Overall accuracy: {}".format(acc))
    acc_dict = {}
    for i in range(len(expected)):
        expected_class = expected[i]
        if expected_class not in acc_dict:
            acc_dict[expected_class] = [0, 0]
        acc_dict[expected_class][1] += 1
        if expected_class == predicted[i]:
            acc_dict[expected_class][0] += 1
    for k,v in acc_dict.items():
        print("Accuracy for class {}: {}".format(k, v[0]/v[1]))
    for average in ["macro", "weighted", "micro"]:
        f1 = f1_score(expected, predicted, average=average)
        print("{} f1 score: {}".format(average, f1))

In [7]:
y_predicted = []
y_random = []
for data, labels in tqdm(test_loader):
    for i in range(len(data)):
        probabilities = []
        for j in range(num_classes):
            probabilities.append(models[j].predict(x=np.array([data[i]]))[0][1])
        best_class = np.argmax(probabilities)
        highest_prob = probabilities[best_class]
        if highest_prob < 0.5:
            y_predicted.append(num_classes)
        else:
            y_predicted.append(best_class)
        y_random.append(np.random.randint(num_classes + 1))

100%|██████████| 110/110 [07:09<00:00,  3.91s/it]


In [8]:
evaluate(y_predicted, y_test)
evaluate(y_random, y_test)

Overall accuracy: 0.6119061247853463
Accuracy for class 2: 0.8669950738916257
Accuracy for class 4: 0.9418604651162791
Accuracy for class 0: 0.8516746411483254
Accuracy for class 3: 0.7807017543859649
Accuracy for class 1: 0.972972972972973
Accuracy for class 5: 0.3519163763066202
macro f1 score: 0.6698090056903326
weighted f1 score: 0.5987305771258042
micro f1 score: 0.6119061247853463
Overall accuracy: 0.16256439610761306
Accuracy for class 2: 0.17733990147783252
Accuracy for class 4: 0.13953488372093023
Accuracy for class 0: 0.16267942583732056
Accuracy for class 3: 0.16666666666666666
Accuracy for class 1: 0.1891891891891892
Accuracy for class 5: 0.1602787456445993
macro f1 score: 0.1411546645788542
weighted f1 score: 0.18388591180544697
micro f1 score: 0.16256439610761306
